In [1]:
from sklearn import preprocessing
from sklearn.neighbors import KNeighborsClassifier
import pandas as pd
import re
import numpy as np
import xgboost as xgb
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split,GridSearchCV
from sklearn.ensemble import RandomForestClassifier, VotingClassifier
from sklearn.metrics import accuracy_score,recall_score,precision_score,f1_score
from sklearn.metrics import mean_squared_error
from sklearn.linear_model import LogisticRegression
import lightgbm as lgb
from catboost import CatBoostClassifier
%matplotlib inline


le = preprocessing.LabelEncoder()


### vamos a aplicar knn a unos score obtenidos por embeddings contextualizados en otros modelos.

#### Traigo mi train set

In [2]:
train = pd.read_csv('csv/train_2_KNN_embeddings.csv')
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7613 entries, 0 to 7612
Data columns (total 10 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   Unnamed: 0      7613 non-null   int64  
 1   id              7613 non-null   int64  
 2   keyword         7552 non-null   object 
 3   location        5080 non-null   object 
 4   text            7613 non-null   object 
 5   target          7613 non-null   int64  
 6   word2vec_score  7613 non-null   float64
 7   elmo_score      7613 non-null   float64
 8   glove_score     7613 non-null   float64
 9   bert_score      7613 non-null   float64
dtypes: float64(4), int64(3), object(3)
memory usage: 594.9+ KB


In [3]:
train.head(10)

,Unnamed: 0,id,keyword,location,text,target,word2vec_score,elmo_score,glove_score,bert_score
0,0,1,NaN,NaN,Our Deeds are the Reason of this #earthquake M...,1,0.556296,0.522046,0.587237,0.567960
1,1,4,NaN,NaN,Forest fire near La Ronge Sask. Canada,1,0.674412,0.538970,0.676435,0.548946
2,2,5,NaN,NaN,All residents asked to 'shelter in place' are ...,1,0.725413,0.583167,0.668918,0.589782
3,3,6,NaN,NaN,"13,000 people receive #wildfires evacuation or...",1,0.758587,0.650540,0.656915,0.602543
4,4,7,NaN,NaN,Just got sent this photo from Ruby #Alaska as ...,1,0.598320,0.570466,0.573899,0.531318
5,5,8,NaN,NaN,#RockyFire Update => California Hwy. 20 closed...,1,0.715882,0.652919,0.756050,0.708653
6,6,10,NaN,NaN,#flood #disaster Heavy rain causes flash flood...,1,0.701084,0.754674,0.785661,0.753023
7,7,13,NaN,NaN,I'm on top of the hill and I can see a fire in...,1,0.418606,0.347374,0.567888,0.468912
8,8,14,NaN,NaN,There's an emergency evacuation happening now ...,1,0.653519,0.509503,0.648205,0.668390
9,9,15,NaN,NaN,I'm afraid that the tornado is coming to our a...,1,0.514508,0.532526,0.579315,0.458969


In [4]:
x = train[['word2vec_score', 'elmo_score', 'glove_score', 'bert_score']]
y = train['target']

In [5]:
x.head()

,word2vec_score,elmo_score,glove_score,bert_score
0,0.556296,0.522046,0.587237,0.567960
1,0.674412,0.538970,0.676435,0.548946
2,0.725413,0.583167,0.668918,0.589782
3,0.758587,0.650540,0.656915,0.602543
4,0.598320,0.570466,0.573899,0.531318


In [6]:
y.head()

0    1
1    1
2    1
3    1
4    1
Name: target, dtype: int64

In [7]:
X_train, X_test, y_train, y_test = train_test_split(x,y, test_size=0.1)

In [8]:
knn = KNeighborsClassifier(n_neighbors=8)
knn.fit(X_train, y_train)
y_pred = knn.predict(X_test)

In [9]:
print('Accuracy Score : ' + str(accuracy_score(y_test,y_pred)))
print('Precision Score : ' + str(precision_score(y_test,y_pred)))
print('Recall Score : ' + str(recall_score(y_test,y_pred)))
print('F1 Score : ' + str(f1_score(y_test,y_pred)))

Accuracy Score : 0.926509186351706
Precision Score : 0.9543859649122807
Recall Score : 0.8634920634920635
F1 Score : 0.9066666666666667


### Como estoy, en KNN no necesito tener test. Genero un knn con un train con todo mi DT. En base a eso genero el target de mi data_test y lo summiteo a kaggle.

In [10]:
train = pd.read_csv('csv/train_2_KNN_embeddings.csv')
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7613 entries, 0 to 7612
Data columns (total 10 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   Unnamed: 0      7613 non-null   int64  
 1   id              7613 non-null   int64  
 2   keyword         7552 non-null   object 
 3   location        5080 non-null   object 
 4   text            7613 non-null   object 
 5   target          7613 non-null   int64  
 6   word2vec_score  7613 non-null   float64
 7   elmo_score      7613 non-null   float64
 8   glove_score     7613 non-null   float64
 9   bert_score      7613 non-null   float64
dtypes: float64(4), int64(3), object(3)
memory usage: 594.9+ KB


In [11]:
x = train[['word2vec_score', 'elmo_score', 'glove_score', 'bert_score']]
y = train['target']

In [12]:
x.head()

,word2vec_score,elmo_score,glove_score,bert_score
0,0.556296,0.522046,0.587237,0.567960
1,0.674412,0.538970,0.676435,0.548946
2,0.725413,0.583167,0.668918,0.589782
3,0.758587,0.650540,0.656915,0.602543
4,0.598320,0.570466,0.573899,0.531318


In [13]:
y.head(6)

0    1
1    1
2    1
3    1
4    1
5    1
Name: target, dtype: int64

In [14]:
knn = KNeighborsClassifier(n_neighbors=5)
knn.fit(x, y)

KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
                     metric_params=None, n_jobs=None, n_neighbors=5, p=2,
                     weights='uniform')

In [15]:
test = pd.read_csv('csv/test_2_KNN_embeddings.csv')
test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3263 entries, 0 to 3262
Data columns (total 9 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   Unnamed: 0      3263 non-null   int64  
 1   id              3263 non-null   int64  
 2   keyword         3237 non-null   object 
 3   location        2158 non-null   object 
 4   text            3263 non-null   object 
 5   word2vec_score  3263 non-null   float64
 6   elmo_score      3263 non-null   float64
 7   glove_score     3263 non-null   float64
 8   bert_score      3263 non-null   float64
dtypes: float64(4), int64(2), object(3)
memory usage: 229.6+ KB


In [16]:
test.head()

,Unnamed: 0,id,keyword,location,text,word2vec_score,elmo_score,glove_score,bert_score
0,0,0,NaN,NaN,Just happened a terrible car crash,0.534202,0.379233,0.574705,0.482696
1,1,2,NaN,NaN,"Heard about #earthquake is different cities, s...",0.583796,0.367322,0.506844,0.520214
2,2,3,NaN,NaN,"there is a forest fire at spot pond, geese are...",0.527839,0.678700,0.411923,0.455055
3,3,9,NaN,NaN,Apocalypse lighting. #Spokane #wildfires,0.548836,0.430616,0.630913,0.642100
4,4,11,NaN,NaN,Typhoon Soudelor kills 28 in China and Taiwan,0.726522,0.584599,0.705246,0.644486


In [17]:
x_test = test [['word2vec_score', 'elmo_score', 'glove_score', 'bert_score']]

In [18]:
y_test = knn.predict(x_test)

In [19]:
y_test

array([0, 1, 1, ..., 1, 1, 1])

In [20]:
resultado = test.copy()

In [21]:
resultado ['target'] = y_test

In [22]:
resultado = resultado [ ['id', 'target'] ]
resultado.head()

,id,target
0,0,0
1,2,1
2,3,1
3,9,1
4,11,1


In [23]:
resultado.set_index('id', inplace=True)

In [24]:
resultado.head(10)

,target
id,
0,0
2,1
3,1
9,1
11,1
12,1
21,0
22,0
27,0


In [25]:
resultado.to_csv("csv/submission_knn_2.csv")